#Используемые данные

https://drive.google.com/drive/folders/1GrUAvxSE6y8WA0Lg5IE_mWwsf3LtHqmR?usp=sharing

# Сохранение данных

Обучение будет длится порядка 9-10 часов, поэтому лучше обезопасить себя от выходок колаба и сохранять все параллельно на диск

In [3]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [5]:
cd /content/drive/MyDrive

/content/drive/MyDrive


#Загрузка библиотеки

In [ ]:
!wget https://github.com/taesungp/contrastive-unpaired-translation/archive/87ab89cdca651f87742844016b0cfa49fa7bd3ee.zip

--2021-11-29 13:54:11--  https://github.com/taesungp/contrastive-unpaired-translation/archive/87ab89cdca651f87742844016b0cfa49fa7bd3ee.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/taesungp/contrastive-unpaired-translation/zip/87ab89cdca651f87742844016b0cfa49fa7bd3ee [following]
--2021-11-29 13:54:11--  https://codeload.github.com/taesungp/contrastive-unpaired-translation/zip/87ab89cdca651f87742844016b0cfa49fa7bd3ee
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘87ab89cdca651f87742844016b0cfa49fa7bd3ee.zip’

87ab89cdca651f87742     [    <=>             ]  16.14M  20.4MB/s    in 0.8s    

2021-11-29 13:54:13 (20.4 MB/s) -

In [ ]:
!unzip -qq 87ab89cdca651f87742844016b0cfa49fa7bd3ee.zip

In [6]:
cd contrastive-unpaired-translation-87ab89cdca651f87742844016b0cfa49fa7bd3ee

/content/drive/MyDrive/contrastive-unpaired-translation-87ab89cdca651f87742844016b0cfa49fa7bd3ee


In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 676 kB 4.9 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=d77932adaaba1de704b9f06794c0b0a8fa9eb4fa841abcfb25f93b104001a8ae
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=8d23b5e32d726aba80569d854dff58fd4f0f4fdba22efdad240d8925d00bde47
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=785cc140c05d19ab372e44c82951f5d478fec0c11e42370d5f28051bad0fe972
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom GPUtil torchfile


# Обучение модели

Для начала нужно добавить в папку datasets папку с изображениями для эксперимента. В ней должно также быть 2 папки - trainA (изображение, откуда переводим) и trainB - изображение, куда переводим

## Делаем из корги маламута

In [ ]:
!python train.py --model sincut  --name dog_party --dataroot ./datasets/dog_party

In [ ]:
!python test.py --model sincut --name dog_party --dataroot ./datasets/dog_party

## Делаем из абстракции свинку пеппу

In [ ]:
!python train.py --model sincut  --name peppa_pig --dataroot ./datasets/peppa_pig

In [ ]:
!python test.py --model sincut --name peppa_pig --dataroot ./datasets/peppa_pig

## Повышаем качество изображения 

In [ ]:
!python train.py --model sincut  --name quality --dataroot ./datasets/quality

In [ ]:
!python test.py --model sincut --name quality --dataroot ./datasets/quality

# Подобные изображения для обученной модели

Увы, путь до изображения захардкоджен в модель, поэтому есть 2 пути 

1) поменять изображение в Трейн А на нужное
2) в папке data в файле singleimage_dataset.py изменить строку 29 

с

self.dir_A = os.path.join(opt.dataroot, 'trainA')

на 

self.dir_A = os.path.join(opt.dataroot, 'testA')

и создать такую папку в соих датасетах для этого эксперимента с нужным изображением

дальше - просто запускаем test.py

In [ ]:
!python test.py --model sincut --name dog_party --dataroot ./datasets/dog_party

# Эксперимент 3

In [ ]:
from packaging import version

In [ ]:
import numpy as np

In [ ]:
#нужно редактировать файл stylegan_networks.py в models

#class StyleGAN2Generator -> forward
 def forward(self, input, layers=[], encode_only=False):
        feat, feats = self.encoder(input, [1], True) # 1 - индекс первого блока резнета
        # print(feats)
        if encode_only:
            return feats
        else:
            fake = self.decoder(feat)
            return fake, feats


In [ ]:
#нужно редактировать файл cut_model.py в models


#class CUTModel -> 

#def __init__

#добавить 'feats_B' в self.visual_names, чтобы потом график построился


#def forward модифицировать для возможности приема вектора признаков

 def forward(self):
        """Run forward pass; called by both functions <optimize_parameters> and <test>."""
        self.real = torch.cat((self.real_A, self.real_B), dim=0) if self.opt.nce_idt and self.opt.isTrain else self.real_A
        if self.opt.flip_equivariance:
            self.flipped_for_equivariance = self.opt.isTrain and (np.random.random() < 0.5)
            if self.flipped_for_equivariance:
                self.real = torch.flip(self.real, [3])

        self.fake, self.feats = self.netG(self.real) #!

        self.fake_B = self.fake[:self.real_A.size(0)]
        self.feats_B = self.feats[0][:self.real_A.size(0)] #!

        if self.opt.nce_idt:
            self.idt_B = self.fake[self.real_A.size(0):]
        
        
        self.fake_a, self.feats_a = self.netG(self.real_B)
        self.feats_A = self.feats_a[0][:self.real_A.size(0)]

## далее извлекаем вектора признаков всех патчей вхрдных данных
        
        base_ind_i = 50
        base_ind_j = 800
        
        vector_base = self.feats_B[:,:,base_ind_i,base_ind_j].view(-1, 1)


        

        result = torch.zeros((1, 3, self.feats_A.size(2), self.feats_A.size(3)))
        
        
        for i in range(self.feats_A.size(2)):
          if i % 25 == 0:
            print(i)
          for j in range(self.feats_A.size(3)):
            vector_local = self.feats_A[:,:, i, j].view(-1, 1)
            if i == 0 and j == 0:
              print(torch.exp(torch.sum(vector_base * vector_local)/0.07))
            result[0][0][i][j] = 1 - torch.exp(torch.sum(vector_base * vector_local)/0.07)
            result[0][1][i][j] = result[0][0][i][j]
            result[0][2][i][j] = result[0][0][i][j]
        print(result)

        base_ind_i = 100
        base_ind_j = 1100

        vector_base = self.feats_A[:,:,base_ind_i,base_ind_j].view(-1, 1)

        result2 = torch.zeros((1, 3, self.feats_B.size(2), self.feats_B.size(3)))
        
        
        for i in range(self.feats_B.size(2)):
          if i % 25 == 0:
            print(i)
          for j in range(self.feats_Bsize(3)):
            vector_local = self.feats_B[:,:, i, j].view(-1, 1)
            if i == 0 and j == 0:
              print(torch.exp(torch.sum(vector_base * vector_local)/0.07))
            result2[0][0][i][j] = 1 - torch.exp(torch.sum(vector_base * vector_local)/0.07)
            result2[0][1][i][j] = result2[0][0][i][j]
            result2[0][2][i][j] = result2[0][0][i][j]



        self.feats_B = result
        self.feats_A = result2
        print(self.feats_B.size())


теперь в папке results в  папке текущего эксперимента  должны появиться папки feats_B и feats_A с нужными изображениями

NB -  если изображение слишком выского разрешения, то при прогоне его через генератор может закончится ОЗУ. советую сразу подбирать более маленькие размеры изображений

In [ ]:
!python test_test.py --model sincut --name dog_party --dataroot ./datasets/dog_party